In [ ]:
# @title
# ==========================================
# HLS.ipynb - Análise de Mata Ciliar com Dados HLS (Harmonized Landsat Sentinel)
# ==========================================
# Este notebook processa dados HLS da NASA para análise de mata ciliar:
# - Busca automática de dados HLS via STAC API
# - Processamento NDVI com máscaras de qualidade
# - Detecção de pontos críticos de degradação
# - Exportação de resultados para integração web
# ==========================================

print("🚀 Iniciando HLS.ipynb - Análise de Mata Ciliar")
print("📡 Processamento de dados HLS (Harmonized Landsat Sentinel)")
print("🌿 Foco: Detecção de degradação em mata ciliar")
print("=" * 60)

# --- 1. Instalação de dependências ---
print("📦 Instalando dependências...")
!pip install pystac-client planetary_computer rasterio rioxarray geopandas shapely matplotlib numpy requests folium contextily stackstac xarray dask -q

# --- 2. Importações principais ---
import pystac_client
import planetary_computer as pc
import rasterio
from rasterio.mask import mask
from rasterio.features import rasterize
from rasterio.transform import from_bounds
import rioxarray as rxr
import geopandas as gpd
from shapely.geometry import shape, Point, box
import matplotlib.pyplot as plt
import numpy as np
import json
import requests
import pandas as pd
import os
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Importações para processamento HLS
import stackstac
import xarray as xr
from pathlib import Path

print("✅ Dependências instaladas e importadas com sucesso!")
print("🔧 Versões principais:")
print(f"   - rasterio: {rasterio.__version__}")
print(f"   - geopandas: {gpd.__version__}")
print(f"   - numpy: {np.__version__}")

# @title
# ==========================================
# ETAPA 1: Carregamento e Preparação da AOI (Área de Interesse)
# ==========================================

print("📍 ETAPA 1: Carregamento da Área de Interesse")
print("-" * 50)

def check_hls_coverage(bounds):
    """Verifica se a região tem cobertura HLS teórica"""
    minx, miny, maxx, maxy = bounds

    # HLS tem cobertura global, mas com algumas limitações
    # Verificações básicas de cobertura
    print("🌍 Verificando cobertura HLS para a região...")

    # Verificar se está dentro dos limites globais razoáveis
    if miny < -60 or maxy > 80:
        print("⚠️ Região pode ter cobertura limitada (latitudes extremas)")
        return False

    # Verificar se a região não é muito pequena
    area_deg = (maxx - minx) * (maxy - miny)
    if area_deg < 0.001:  # Muito pequena
        print("⚠️ Região muito pequena - expandindo ligeiramente...")
        return False

    # Verificar se não é muito grande
    if area_deg > 10:  # Muito grande
        print("⚠️ Região muito grande - pode ser necessário dividir")
        return False

    print("✅ Região dentro dos parâmetros esperados para HLS")
    return True

# Configurações
AOI_FILE = "export.geojson"  # Arquivo padrão do projeto
BUFFER_DISTANCE = 200  # Buffer de mata ciliar em metros
CRS_WGS84 = "EPSG:4326"

def load_aoi_data():
    """Carrega dados da AOI com múltiplas opções de fonte"""

    # Opção 1: Tentar carregar arquivo local do projeto
    local_paths = [
        "../../export.geojson",
        "../data/export.geojson",
        "export.geojson",
        "../scripts/data/export.geojson"
    ]

    for path in local_paths:
        if os.path.exists(path):
            print(f"📂 Carregando AOI local: {path}")
            with open(path) as f:
                data = json.load(f)
            gdf = gpd.GeoDataFrame.from_features(data["features"], crs=CRS_WGS84)
            return gdf, path

    # Opção 2: Upload manual (Google Colab)
    try:
        from google.colab import files
        print("📤 Arquivo local não encontrado. Faça upload do arquivo GeoJSON:")
        uploaded = files.upload()
        geojson_path = list(uploaded.keys())[0]

        with open(geojson_path) as f:
            data = json.load(f)
        gdf = gpd.GeoDataFrame.from_features(data["features"], crs=CRS_WGS84)
        return gdf, geojson_path

    except ImportError:
        # Opção 3: AOI de exemplo (Sinimbu/RS)
        print("⚠️ Usando AOI de exemplo - Sinimbu/RS")
        example_coords = [
            [-52.5, -29.4], [-52.4, -29.4],
            [-52.4, -29.5], [-52.5, -29.5], [-52.5, -29.4]
        ]
        from shapely.geometry import Polygon
        example_geom = Polygon(example_coords)
        gdf = gpd.GeoDataFrame([1], geometry=[example_geom], crs=CRS_WGS84)
        return gdf, "exemplo_sinimbu"

# Carregar AOI
try:
    aoi_gdf, source_path = load_aoi_data()

    # Validar e preparar AOI
    print(f"✅ AOI carregada: {source_path}")
    print(f"📊 Informações da AOI:")
    print(f"   - Geometrias: {len(aoi_gdf)}")
    print(f"   - Tipo: {aoi_gdf.geometry.geom_type.iloc[0]}")
    print(f"   - CRS: {aoi_gdf.crs}")
    print(f"   - Bounds: {aoi_gdf.total_bounds}")

    # Criar buffer para mata ciliar
    print(f"🌊 Criando buffer de mata ciliar ({BUFFER_DISTANCE}m)...")

    # Converter para UTM para buffer em metros
    # Estimar zona UTM baseada no centróide
    centroid = aoi_gdf.geometry.centroid.iloc[0]
    utm_zone = int((centroid.x + 180) / 6) + 1
    utm_crs = f"EPSG:{32700 + utm_zone}" if centroid.y < 0 else f"EPSG:{32600 + utm_zone}"

    print(f"🗺️ Convertendo para UTM: {utm_crs}")
    aoi_utm = aoi_gdf.to_crs(utm_crs)
    aoi_buffer_utm = aoi_utm.buffer(BUFFER_DISTANCE)
    aoi_buffer_gdf = gpd.GeoDataFrame(geometry=aoi_buffer_utm, crs=utm_crs).to_crs(CRS_WGS84)

    # Calcular área total
    area_km2 = aoi_buffer_utm.area.sum() / 1_000_000
    print(f"📏 Área total com buffer: {area_km2:.2f} km²")

    # Definir bounds para busca HLS
    bounds = aoi_buffer_gdf.total_bounds  # [minx, miny, maxx, maxy]
    print(f"🎯 Bounds para busca HLS: {bounds}")

    # Verificar cobertura HLS
    hls_coverage_ok = check_hls_coverage(bounds)
    if not hls_coverage_ok:
        print("⚠️ Região pode ter cobertura HLS limitada")

except Exception as e:
    print(f"❌ Erro ao carregar AOI: {e}")
    raise

# @title
# ==========================================
# ETAPA 2: Busca e Download de Dados HLS via STAC API
# ==========================================

print("\n📡 ETAPA 2: Busca de Dados HLS")
print("-" * 50)

# Configurações de busca - CORRIGIDAS para Microsoft Planetary Computer
START_DATE = "2022-06-01"  # Período com dados HLS confirmados
END_DATE = "2022-09-30"    # Fim do período (verão/outono)
CLOUD_COVERAGE_MAX = 50    # Aumentado para 50% para encontrar mais dados

# Nomes corretos das coleções HLS no Microsoft Planetary Computer
HLS_COLLECTIONS = [
    "hls2-l30",  # HLS Landsat 30m v2.0 (encontrado no diagnóstico)
    "hls2-s30"   # HLS Sentinel-2 30m v2.0 (encontrado no diagnóstico)
]

print("🔧 CONFIGURAÇÕES AJUSTADAS:")
print(f"   📅 Período ajustado para: {START_DATE} a {END_DATE}")
print(f"   ☁️ Tolerância de nuvens: {CLOUD_COVERAGE_MAX}%")
print("   💡 Motivo: Garantir disponibilidade de dados HLS")
print("")
print("📡 SOBRE OS DADOS HLS:")
print("   🛰️ Harmonized Landsat Sentinel-2 (HLS)")
print("   📏 Resolução: 30 metros")
print("   🔄 Revisita: 2-3 dias (combinando Landsat + Sentinel-2)")
print("   🌍 Cobertura: Global")
print("   📅 Disponível desde: 2013 (Landsat) / 2015 (Sentinel-2)")
print("   🏢 Hospedado por: Microsoft Planetary Computer")
print("   🔗 Fonte: NASA LP DAAC")

def search_hls_data(bounds, start_date, end_date, max_cloud=50):
    """Busca dados HLS via Microsoft Planetary Computer STAC API com diagnósticos avançados"""

    print(f"🔍 Buscando dados HLS...")
    print(f"   📅 Período: {start_date} a {end_date}")
    print(f"   ☁️ Máx. nuvens: {max_cloud}%")
    print(f"   📍 Bounds: {bounds}")

    try:
        # Conectar ao catálogo STAC do Microsoft Planetary Computer
        print("🌐 Conectando ao Microsoft Planetary Computer...")
        print("   📡 Fonte: NASA HLS data via Microsoft Azure")
        print("   🔗 Referência: https://www.earthdata.nasa.gov/news/blog/harmonized-landsat-sentinel-2-hls-data-now-available-microsofts-planetary-computer")

        catalog = pystac_client.Client.open(
            "https://planetarycomputer.microsoft.com/api/stac/v1",
            modifier=pc.sign_inplace
        )
        print("✅ Conexão estabelecida com Microsoft Planetary Computer")

        # Verificar coleções HLS disponíveis no Microsoft Planetary Computer
        print("📋 Verificando coleções HLS no Microsoft Planetary Computer...")
        print("   🔍 Buscando: HLS Landsat 30m e HLS Sentinel-2 30m")

        available_collections = []

        # Primeiro, listar todas as coleções para debug
        try:
            all_collections = list(catalog.get_collections())
            hls_related = [c.id for c in all_collections if 'hls' in c.id.lower()]
            if hls_related:
                print(f"   🔍 Coleções HLS encontradas: {hls_related}")
            else:
                print("   ⚠️ Nenhuma coleção HLS encontrada, listando todas disponíveis...")
                all_ids = [c.id for c in all_collections[:10]]  # Primeiras 10
                print(f"   📋 Primeiras coleções: {all_ids}")
        except Exception as e:
            print(f"   ⚠️ Erro ao listar coleções: {e}")

        # Usar os nomes corretos encontrados no diagnóstico
        for collection_id in HLS_COLLECTIONS:
            try:
                collection = catalog.get_collection(collection_id)
                available_collections.append(collection_id)
                print(f"   ✅ {collection_id}: Disponível")

                # Mostrar informações detalhadas
                if hasattr(collection, 'extent') and collection.extent.temporal:
                    temporal_extent = collection.extent.temporal.intervals[0]
                    print(f"      📅 Período: {temporal_extent[0]} a {temporal_extent[1]}")

                if hasattr(collection, 'description'):
                    desc = collection.description[:100] + "..." if len(collection.description) > 100 else collection.description
                    print(f"      📝 Descrição: {desc}")

            except Exception as e:
                print(f"   ❌ {collection_id}: Não disponível ({e})")

        if not available_collections:
            print("❌ Nenhuma coleção HLS disponível!")
            return None

        # Buscar itens HLS
        all_items = []

        for collection in available_collections:
            print(f"\n🛰️ Buscando coleção: {collection}")

            try:
                # Busca inicial sem filtro de nuvens
                print("   🔍 Busca inicial (sem filtro de nuvens)...")
                search_initial = catalog.search(
                    collections=[collection],
                    bbox=bounds,
                    datetime=f"{start_date}/{end_date}"
                )

                initial_items = list(search_initial.items())
                print(f"   📊 Total de itens no período: {len(initial_items)}")

                if len(initial_items) == 0:
                    print("   ⚠️ Nenhum item encontrado no período especificado")

                    # Tentar período mais amplo
                    print("   🔄 Tentando período mais amplo (2021-2023)...")
                    search_extended = catalog.search(
                        collections=[collection],
                        bbox=bounds,
                        datetime="2021-01-01/2023-12-31"
                    )

                    extended_items = list(search_extended.items())
                    print(f"   📊 Itens no período estendido: {len(extended_items)}")

                    if len(extended_items) > 0:
                        print("   💡 Dados disponíveis em outros períodos:")
                        for item in extended_items[:3]:
                            date = item.properties.get("datetime", "N/A")[:10]
                            cloud = item.properties.get("eo:cloud_cover", "N/A")
                            print(f"      - {date} | ☁️ {cloud}%")
                    continue

                # Aplicar filtro de nuvens
                print(f"   ☁️ Aplicando filtro de nuvens (< {max_cloud}%)...")
                filtered_items = []

                for item in initial_items:
                    cloud_cover = item.properties.get("eo:cloud_cover", 100)
                    if cloud_cover < max_cloud:
                        filtered_items.append(item)

                print(f"   ✅ Itens após filtro: {len(filtered_items)}")
                all_items.extend(filtered_items)

                # Mostrar alguns exemplos
                if filtered_items:
                    print("   🏆 Melhores itens desta coleção:")
                    sorted_items = sorted(filtered_items, key=lambda x: x.properties.get("eo:cloud_cover", 100))
                    for i, item in enumerate(sorted_items[:3]):
                        cloud_cover = item.properties.get("eo:cloud_cover", "N/A")
                        date = item.properties.get("datetime", "N/A")[:10]
                        print(f"      {i+1}. {date} | ☁️ {cloud_cover}%")

            except Exception as e:
                print(f"   ❌ Erro na busca {collection}: {e}")
                import traceback
                print(f"   🔍 Detalhes: {traceback.format_exc()}")
                continue

        if not all_items:
            print("\n❌ DIAGNÓSTICO: Nenhum item HLS encontrado!")
            print("🔍 Possíveis causas:")
            print("   1. Região fora da cobertura HLS")
            print("   2. Período sem dados disponíveis")
            print("   3. Filtros muito restritivos")
            print("   4. Problemas de conectividade")

            print("\n💡 Sugestões:")
            print("   - Tente um período diferente (2021-2022)")
            print("   - Aumente a tolerância de nuvens")
            print("   - Verifique se a região tem cobertura HLS")
            return None

        # Ordenar por cobertura de nuvens
        all_items.sort(key=lambda x: x.properties.get("eo:cloud_cover", 100))

        print(f"\n📊 RESULTADO FINAL:")
        print(f"   ✅ Total de itens encontrados: {len(all_items)}")
        print(f"   🏆 Melhores cenas (menor cobertura de nuvens):")

        for i, item in enumerate(all_items[:5]):
            cloud_cover = item.properties.get("eo:cloud_cover", "N/A")
            date = item.properties.get("datetime", "N/A")[:10]
            collection = item.collection_id
            print(f"      {i+1}. {collection} | {date} | ☁️ {cloud_cover}%")

        return all_items

    except Exception as e:
        print(f"❌ Erro crítico na busca HLS: {e}")
        import traceback
        print(f"🔍 Traceback completo:")
        print(traceback.format_exc())
        return None

def select_best_item(items, max_items=3):
    """Seleciona os melhores itens HLS baseado em critérios de qualidade"""

    if not items:
        return None

    print(f"\n🎯 Selecionando melhores itens (máx: {max_items})...")

    # Filtrar e ordenar
    filtered_items = []

    for item in items:
        cloud_cover = item.properties.get("eo:cloud_cover", 100)
        date = item.properties.get("datetime", "")

        # Critérios de seleção
        if cloud_cover <= CLOUD_COVERAGE_MAX:
            filtered_items.append({
                'item': item,
                'cloud_cover': cloud_cover,
                'date': date,
                'score': 100 - cloud_cover  # Score simples baseado em nuvens
            })

    # Ordenar por score
    filtered_items.sort(key=lambda x: x['score'], reverse=True)

    # Selecionar os melhores
    selected_items = filtered_items[:max_items]

    print("✅ Itens selecionados:")
    for i, sel in enumerate(selected_items):
        item = sel['item']
        print(f"   {i+1}. {item.collection_id} | {sel['date'][:10]} | ☁️ {sel['cloud_cover']}%")

    return [sel['item'] for sel in selected_items]

def try_alternative_periods_and_regions(bounds, original_start, original_end):
    """Tenta períodos e configurações alternativas para encontrar dados HLS"""

    print("\n🔄 TENTANDO ALTERNATIVAS...")

    # Períodos alternativos conhecidos com dados HLS
    alternative_periods = [
        ("2022-01-01", "2022-12-31", "Ano completo 2022"),
        ("2021-06-01", "2021-09-30", "Verão 2021"),
        ("2023-01-01", "2023-06-30", "Primeiro semestre 2023"),
        ("2020-06-01", "2020-09-30", "Verão 2020")
    ]

    for start_date, end_date, description in alternative_periods:
        print(f"\n🔍 Tentando período: {description} ({start_date} a {end_date})")

        try:
            items = search_hls_data(bounds, start_date, end_date, CLOUD_COVERAGE_MAX)
            if items and len(items) > 0:
                print(f"✅ SUCESSO! Encontrados {len(items)} itens no período {description}")
                return items, start_date, end_date
        except Exception as e:
            print(f"❌ Falhou para {description}: {e}")
            continue

    # Se ainda não encontrou, tentar região ligeiramente expandida
    print("\n🗺️ Tentando região expandida...")
    minx, miny, maxx, maxy = bounds
    buffer = 0.1  # ~11km de buffer
    expanded_bounds = [minx - buffer, miny - buffer, maxx + buffer, maxy + buffer]

    try:
        items = search_hls_data(expanded_bounds, "2022-01-01", "2022-12-31", 70)  # Mais tolerante
        if items and len(items) > 0:
            print(f"✅ SUCESSO com região expandida! Encontrados {len(items)} itens")
            return items, "2022-01-01", "2022-12-31"
    except Exception as e:
        print(f"❌ Região expandida também falhou: {e}")

    return None, None, None

# Executar busca com fallbacks inteligentes
try:
    print("🚀 Iniciando busca HLS...")
    hls_items = search_hls_data(bounds, START_DATE, END_DATE, CLOUD_COVERAGE_MAX)
    used_start_date = START_DATE
    used_end_date = END_DATE

    # Se não encontrou dados, tentar alternativas
    if not hls_items or len(hls_items) == 0:
        print("\n🔄 Dados não encontrados no período original, tentando alternativas...")
        hls_items, used_start_date, used_end_date = try_alternative_periods_and_regions(
            bounds, START_DATE, END_DATE
        )

    if hls_items and len(hls_items) > 0:
        selected_hls_items = select_best_item(hls_items, max_items=3)
        print(f"\n✅ {len(selected_hls_items)} itens HLS selecionados para processamento")
        print(f"📅 Período utilizado: {used_start_date} a {used_end_date}")

        # Atualizar variáveis globais para uso posterior
        START_DATE = used_start_date
        END_DATE = used_end_date

    else:
        print("\n❌ FALHA TOTAL: Nenhum item HLS encontrado")
        print("🔍 Possíveis soluções:")
        print("   1. Verificar se a região tem cobertura HLS")
        print("   2. Usar dados Sentinel-2 diretamente")
        print("   3. Tentar uma região diferente")
        print("   4. Verificar conectividade com Microsoft Planetary Computer")
        selected_hls_items = None

except Exception as e:
    print(f"❌ Erro crítico na busca HLS: {e}")
    import traceback
    print("🔍 Traceback completo:")
    print(traceback.format_exc())
    selected_hls_items = None

# Função para converter tipos NumPy para tipos Python nativos
def convert_numpy_types(obj):
    """Converte tipos NumPy para tipos Python nativos recursivamente"""
    if isinstance(obj, dict):
        return {key: convert_numpy_types(value) for key, value in obj.items()}
    elif isinstance(obj, list):
        return [convert_numpy_types(item) for item in obj]
    elif isinstance(obj, np.integer):
        return int(obj)
    elif isinstance(obj, np.floating):
        return float(obj)
    elif isinstance(obj, np.ndarray):
        return obj.tolist()
    else:
        return obj

# Executar análise (assumindo que ndvi, final_transform, final_crs, gdf_buffer existem)
print("\n🔍 Executando análise com thresholds FIXOS...")
# critical_analysis_fixed = identify_critical_points_fixed_thresholds(ndvi, final_transform, final_crs, gdf_buffer)

# Salvar GeoJSON com correção de tipos
def save_corrected_geojson(analysis_data, filename):
    """Salva GeoJSON com tipos corrigidos"""
    if not analysis_data:
        print("❌ Nenhum dado para salvar")
        return

    features = []

    # Adicionar pontos críticos
    for point in analysis_data['critical']:
        features.append({
            "type": "Feature",
            "geometry": {
                "type": "Point",
                "coordinates": [point['lon'], point['lat']]
            },
            "properties": {
                "severity": point['severity'],
                "ndvi": point['ndvi'],
                "description": point['description'],
                "type": "critical_point"
            }
        })

    # Adicionar pontos moderados
    for point in analysis_data['moderate']:
        features.append({
            "type": "Feature",
            "geometry": {
                "type": "Point",
                "coordinates": [point['lon'], point['lat']]
            },
            "properties": {
                "severity": point['severity'],
                "ndvi": point['ndvi'],
                "description": point['description'],
                "type": "moderate_point"
            }
        })

    # Criar GeoJSON
    geojson_data = {
        "type": "FeatureCollection",
        "features": features,
        "metadata": {
            "analysis_date": pd.Timestamp.now().isoformat(),
            "buffer_distance": "500 meters each side",
            "thresholds": analysis_data['thresholds'],
            "total_critical_points": len(analysis_data['critical']),
            "total_moderate_points": len(analysis_data['moderate']),
            "method": "fixed_thresholds_corrected"
        }
    }

    # Converter tipos e salvar
    geojson_clean = convert_numpy_types(geojson_data)

    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(geojson_clean, f, indent=2, ensure_ascii=False)

    print(f"✅ GeoJSON salvo: {filename}")
    print(f"📊 Total de pontos: {len(features)}")

print("\n🎯 Notebook corrigido e pronto para uso!")
print("📋 Para usar:")
print("1. Execute as células anteriores para carregar dados NDVI")
print("2. Execute esta célula para análise com thresholds fixos")
print("3. Use save_corrected_geojson(critical_analysis_fixed, 'output.geojson')")

# @title
# ==========================================
# ETAPA 3: Processamento NDVI com Máscaras de Qualidade
# ==========================================

print("\n🌿 ETAPA 3: Processamento NDVI")
print("-" * 50)

# Configurações NDVI
NDVI_CRITICAL_THRESHOLD = 0.2   # Limite para áreas sem vegetação ou muito ralas
NDVI_MODERATE_THRESHOLD = 0.5   # Limite para separar vegetação esparsa de densa
MIN_VALID_PIXELS = 0.05         # Mínimo 5% de pixels válidos

def load_and_process_hls_data(item, aoi_bounds):
    """Carrega e processa dados HLS para cálculo NDVI"""

    print(f"📥 Processando item: {item.collection_id}")
    print(f"   📅 Data: {item.properties.get('datetime', 'N/A')[:10]}")
    print(f"   ☁️ Nuvens: {item.properties.get('eo:cloud_cover', 'N/A')}%")

    try:
        # Definir bandas necessárias para NDVI - HLS v2.0
        if "hls2-l30" in item.collection_id:  # HLS Landsat
            red_band = "B04"    # Red (665nm)
            nir_band = "B05"    # NIR (865nm) - Landsat usa B05 para NIR
            qa_band = "Fmask"   # Quality mask
            print(f"   🛰️ Tipo: HLS Landsat (L30)")
        else:  # HLS Sentinel-2
            red_band = "B04"    # Red (665nm)
            nir_band = "B08"    # NIR (842nm) - Sentinel-2 usa B08 para NIR
            qa_band = "Fmask"   # Quality mask
            print(f"   🛰️ Tipo: HLS Sentinel-2 (S30)")

        bands_to_load = [red_band, nir_band, qa_band]
        print(f"   📊 Bandas: {bands_to_load}")

        # Carregar dados HLS usando abordagem alternativa (rioxarray)
        print(f"   📥 Carregando dados HLS...")

        # Determinar CRS dos dados HLS baseado no tile
        item_id = item.id
        print(f"   🗺️ Item ID: {item_id}")

        # Extrair zona UTM do tile HLS
        import re
        tile_match = re.search(r'T(\d{2})([A-Z])', item_id)

        if tile_match:
            zone_num = int(tile_match.group(1))
            lat_band = tile_match.group(2)

            if lat_band < 'N':
                hls_crs = f"EPSG:{32700 + zone_num}"
                hemisphere = "Sul"
            else:
                hls_crs = f"EPSG:{32600 + zone_num}"
                hemisphere = "Norte"

            print(f"   🌍 Zona UTM: {zone_num}{lat_band} ({hemisphere})")
        else:
            hls_crs = "EPSG:32722"
            print(f"   ⚠️ Usando CRS padrão: 22S")

        print(f"   🗺️ CRS detectado: {hls_crs}")

        # Abordagem alternativa: carregar bandas individualmente
        try:
            print("   🔄 Carregando bandas individualmente...")

            # Carregar cada banda separadamente
            band_arrays = {}

            for band in bands_to_load:
                print(f"      📊 Carregando banda {band}...")

                # Obter URL da banda
                band_asset = item.assets[band]
                band_url = band_asset.href

                # Assinar URL se necessário (Microsoft Planetary Computer)
                if 'planetarycomputer' in band_url:
                    band_url = pc.sign(band_url)

                # Carregar com rioxarray
                try:
                    band_data = rxr.open_rasterio(
                        band_url,
                        chunks={'x': 512, 'y': 512}  # Chunking para performance
                    )

                    # Reprojetar para bounds se necessário
                    if str(band_data.rio.crs) != "EPSG:4326":
                        # Reprojetar bounds para CRS da banda
                        from pyproj import Transformer
                        transformer = Transformer.from_crs("EPSG:4326", band_data.rio.crs, always_xy=True)
                        minx_proj, miny_proj = transformer.transform(aoi_bounds[0], aoi_bounds[1])
                        maxx_proj, maxy_proj = transformer.transform(aoi_bounds[2], aoi_bounds[3])

                        # Recortar para AOI
                        band_data = band_data.rio.clip_box(minx_proj, miny_proj, maxx_proj, maxy_proj)
                    else:
                        # Recortar diretamente
                        band_data = band_data.rio.clip_box(*aoi_bounds)

                    band_arrays[band] = band_data.squeeze()
                    print(f"      ✅ {band}: {band_data.shape}")

                except Exception as band_error:
                    print(f"      ❌ Erro ao carregar {band}: {band_error}")
                    raise band_error

            # Verificar se todas as bandas foram carregadas
            if len(band_arrays) != len(bands_to_load):
                raise Exception("Nem todas as bandas foram carregadas")

            # Extrair bandas individuais
            red = band_arrays[red_band]
            nir = band_arrays[nir_band]
            qa = band_arrays[qa_band]

            print(f"   ✅ Todas as bandas carregadas com sucesso!")
            print(f"   📐 Dimensões Red: {red.shape}")
            print(f"   📐 Dimensões NIR: {nir.shape}")
            print(f"   📐 Dimensões QA: {qa.shape}")
            print(f"   🗺️ CRS: {red.rio.crs}")

        except Exception as load_error:
            print(f"   ❌ Erro no carregamento alternativo: {load_error}")
            raise load_error

        # Bandas já foram extraídas na seção anterior
        # red, nir, qa já estão disponíveis

        print(f"   📊 Dimensões das bandas:")
        print(f"      - Red: {red.shape}, dtype: {red.dtype}")
        print(f"      - NIR: {nir.shape}, dtype: {nir.dtype}")
        print(f"      - QA: {qa.shape}, dtype: {qa.dtype}")

        # Aplicar escala HLS se necessário (dados HLS já vêm em reflectância 0-1)
        # Verificar se os dados precisam de escalonamento
        red_max = float(np.nanmax(red.values))
        nir_max = float(np.nanmax(nir.values))

        print(f"   📈 Valores máximos: Red={red_max:.3f}, NIR={nir_max:.3f}")

        # Se os valores estão acima de 1, aplicar escala (dados em 0-10000)
        if red_max > 1.5 or nir_max > 1.5:
            print("   🔢 Aplicando escala HLS (dividindo por 10000)...")
            red = red / 10000.0
            nir = nir / 10000.0

            # Verificar valores após escala
            red_scaled_max = float(np.nanmax(red.values))
            nir_scaled_max = float(np.nanmax(nir.values))
            print(f"   📊 Valores após escala: Red={red_scaled_max:.3f}, NIR={nir_scaled_max:.3f}")
        else:
            print("   ✅ Dados já estão em escala de reflectância (0-1)")

        # Verificar se há valores válidos nas bandas
        red_valid = np.sum(np.isfinite(red.values) & (red.values > 0))
        nir_valid = np.sum(np.isfinite(nir.values) & (nir.values > 0))
        print(f"   📊 Pixels com valores válidos: Red={red_valid}, NIR={nir_valid}")

        # Aplicar máscara de qualidade (Fmask HLS)
        # Fmask HLS: 0=clear, 1=water, 2=cloud_shadow, 3=snow, 4=cloud, 255=fill
        print("   🎭 Aplicando máscara de qualidade...")

        # Primeiro, vamos investigar os valores da máscara QA
        qa_unique = np.unique(qa.values)
        qa_counts = {val: int(np.sum(qa.values == val)) for val in qa_unique}
        print(f"   📊 Valores únicos na máscara QA: {qa_unique}")
        print(f"   📈 Contagem por valor: {qa_counts}")

        # Máscara mais permissiva inicialmente para diagnóstico
        # Incluir: 0=clear, 1=water, 2=cloud_shadow (pode ter vegetação)
        valid_mask = (qa == 0) | (qa == 1) | (qa == 2)

        # Se ainda não há pixels válidos, ser ainda mais permissivo
        valid_pixels_count = int(np.sum(valid_mask.values))
        print(f"   🔍 Pixels válidos com máscara padrão: {valid_pixels_count}")

        if valid_pixels_count == 0:
            print("   ⚠️ Nenhum pixel válido com máscara padrão, tentando máscara mais permissiva...")
            # Incluir tudo exceto fill values (255) e nuvens densas (4)
            valid_mask = (qa != 255) & (qa != 4)
            valid_pixels_count = int(np.sum(valid_mask.values))
            print(f"   🔍 Pixels válidos com máscara permissiva: {valid_pixels_count}")

        if valid_pixels_count == 0:
            print("   ⚠️ Ainda sem pixels válidos, usando todos os pixels para diagnóstico...")
            valid_mask = qa >= 0  # Todos os pixels
            valid_pixels_count = int(np.sum(valid_mask.values))
            print(f"   🔍 Total de pixels: {valid_pixels_count}")

        # Calcular NDVI
        print("   🧮 Calculando NDVI...")
        denominator = nir + red

        # Criar máscara combinada de forma segura
        print("   🎭 Criando máscara combinada...")

        # Componentes da máscara
        denom_ok = (denominator != 0)
        red_ok = (red >= 0) & np.isfinite(red)
        nir_ok = (nir >= 0) & np.isfinite(nir)

        # Diagnósticos das máscaras
        print(f"   📊 Diagnóstico das máscaras:")
        print(f"      - Denominador OK: {int(np.sum(denom_ok.values))}")
        print(f"      - Red OK: {int(np.sum(red_ok.values))}")
        print(f"      - NIR OK: {int(np.sum(nir_ok.values))}")
        print(f"      - QA válido: {valid_pixels_count}")

        # Máscara combinada
        combined_mask = denom_ok & valid_mask & red_ok & nir_ok
        combined_valid = int(np.sum(combined_mask.values))
        print(f"      - Máscara combinada: {combined_valid}")

        # Calcular NDVI com máscara segura
        ndvi = xr.where(
            combined_mask,
            (nir - red) / denominator,
            np.nan
        )

        # Se ainda não há pixels válidos, tentar NDVI sem máscara QA
        if combined_valid == 0:
            print("   🔄 Tentando NDVI sem máscara de qualidade...")
            simple_mask = denom_ok & red_ok & nir_ok
            simple_valid = int(np.sum(simple_mask.values))
            print(f"   📊 Pixels válidos sem QA: {simple_valid}")

            if simple_valid > 0:
                ndvi = xr.where(
                    simple_mask,
                    (nir - red) / denominator,
                    np.nan
                )
                combined_valid = simple_valid

        # Estatísticas
        valid_pixels = np.sum(~np.isnan(ndvi.values))
        total_pixels = ndvi.size
        valid_fraction = valid_pixels / total_pixels

        print(f"   📊 Estatísticas NDVI:")
        print(f"      - Pixels válidos: {valid_pixels:,} ({valid_fraction:.1%})")
        print(f"      - NDVI min: {np.nanmin(ndvi.values):.3f}")
        print(f"      - NDVI max: {np.nanmax(ndvi.values):.3f}")
        print(f"      - NDVI médio: {np.nanmean(ndvi.values):.3f}")

        # Critério mais flexível para áreas pequenas
        min_threshold = min(MIN_VALID_PIXELS, 0.01)  # Pelo menos 1% ou o mínimo configurado

        if valid_fraction < min_threshold:
            print(f"   ⚠️ Poucos pixels válidos ({valid_fraction:.1%} < {min_threshold:.1%})")

            # Se há pelo menos alguns pixels, continuar mesmo assim
            if valid_pixels > 10:  # Pelo menos 10 pixels
                print(f"   💡 Continuando com {valid_pixels} pixels para análise...")
            else:
                print(f"   ❌ Insuficiente para análise (apenas {valid_pixels} pixels)")
                return None

        return {
            'ndvi': ndvi,
            'red': red,
            'nir': nir,
            'qa': qa,
            'valid_mask': valid_mask,
            'valid_fraction': valid_fraction,
            'item': item,
            'stats': {
                'min': float(np.nanmin(ndvi.values)),
                'max': float(np.nanmax(ndvi.values)),
                'mean': float(np.nanmean(ndvi.values)),
                'std': float(np.nanstd(ndvi.values)),
                'valid_pixels': int(valid_pixels),
                'total_pixels': int(total_pixels)
            }
        }

    except Exception as e:
        print(f"   ❌ Erro no processamento: {e}")
        return None

def create_ndvi_composite(processed_items):
    """Cria composição NDVI a partir de múltiplos itens"""

    if not processed_items:
        return None

    print(f"\n🎨 Criando composição NDVI de {len(processed_items)} itens...")

    # Se apenas um item, retornar diretamente
    if len(processed_items) == 1:
        return processed_items[0]

    # Múltiplos itens: criar composição baseada na qualidade
    ndvi_arrays = []
    weights = []

    for item_data in processed_items:
        ndvi_arrays.append(item_data['ndvi'])
        # Peso baseado na fração de pixels válidos
        weights.append(item_data['valid_fraction'])

    # Composição ponderada
    ndvi_stack = xr.concat(ndvi_arrays, dim='time')
    weights_array = xr.DataArray(weights, dims=['time'])

    # Média ponderada ignorando NaN
    composite_ndvi = ndvi_stack.weighted(weights_array).mean(dim='time', skipna=True)

    print("✅ Composição NDVI criada")
    print(f"   📊 NDVI final min: {np.nanmin(composite_ndvi.values):.3f}")
    print(f"   📊 NDVI final max: {np.nanmax(composite_ndvi.values):.3f}")
    print(f"   📊 NDVI final médio: {np.nanmean(composite_ndvi.values):.3f}")

    return {
        'ndvi': composite_ndvi,
        'source_items': processed_items,
        'stats': {
            'min': float(np.nanmin(composite_ndvi.values)),
            'max': float(np.nanmax(composite_ndvi.values)),
            'mean': float(np.nanmean(composite_ndvi.values)),
            'std': float(np.nanstd(composite_ndvi.values))
        }
    }

# Processar itens HLS selecionados
processed_hls_data = []

if selected_hls_items:
    print("🚀 Processando itens HLS selecionados...")

    for i, item in enumerate(selected_hls_items):
        print(f"\n📊 Processando item {i+1}/{len(selected_hls_items)}...")

        processed_data = load_and_process_hls_data(item, bounds)

        if processed_data:
            processed_hls_data.append(processed_data)
            print("   ✅ Processamento concluído")
        else:
            print("   ❌ Processamento falhou")

    # Criar composição final
    if processed_hls_data:
        final_ndvi_data = create_ndvi_composite(processed_hls_data)
        print(f"\n✅ Processamento NDVI concluído com {len(processed_hls_data)} itens")
    else:
        print("\n❌ Nenhum item HLS processado com sucesso")
        final_ndvi_data = None
else:
    print("❌ Nenhum item HLS disponível para processamento")
    final_ndvi_data = None

# @title
# ==========================================
# ETAPA 4: Análise de Mata Ciliar e Classificação de Severidade
# ==========================================

print("\n🌊 ETAPA 4: Análise de Mata Ciliar")
print("-" * 50)

def classify_vegetation_degradation(ndvi_value):
    """Classifica a cobertura/condição da vegetação baseada no NDVI (escala atualizada)"""

    if np.isnan(ndvi_value):
        return {
            'level': 'no_data',
            'color': '#808080',
            'label': 'Sem Dados',
            'severity': 'unknown'
        }
    elif ndvi_value < 0.0:
        return {
            'level': 'non_vegetated',
            'color': '#0066CC',
            'label': 'Sem vegetação (água/nuvem/neve/rocha)',
            'severity': 'non_vegetated'
        }
    elif ndvi_value < 0.2:
        return {
            'level': 'very_sparse',
            'color': '#DC143C',
            'label': 'Vegetação muito rala / solo exposto',
            'severity': 'very_sparse'
        }
    elif ndvi_value < 0.5:
        return {
            'level': 'sparse',
            'color': '#FF8C00',
            'label': 'Vegetação esparsa / em regeneração',
            'severity': 'sparse'
        }
    elif ndvi_value < 0.8:
        return {
            'level': 'dense',
            'color': '#228B22',
            'label': 'Vegetação densa e saudável',
            'severity': 'dense'
        }
    else:
        return {
            'level': 'extremely_dense',
            'color': '#006400',
            'label': 'Cobertura extremamente densa (rara)',
            'severity': 'extremely_dense'
        }

def analyze_riparian_forest_degradation(ndvi_data, aoi_buffer_gdf):
    """Analisa degradação da mata ciliar dentro do buffer"""

    if not ndvi_data or 'ndvi' not in ndvi_data:
        print("❌ Dados NDVI não disponíveis para análise")
        return None

    ndvi_array = ndvi_data['ndvi']
    print(f"📊 Analisando degradação da mata ciliar...")
    print(f"   📐 Dimensões NDVI: {ndvi_array.shape}")

    # Converter buffer para o CRS do NDVI
    buffer_crs = ndvi_array.rio.crs
    if buffer_crs is None:
        print("❌ NDVI array não possui CRS definido! Tentando definir manualmente...")

        # Verificar se temos dados de origem com CRS
        if 'source_items' in ndvi_data and len(ndvi_data['source_items']) > 0:
            # Usar CRS do primeiro item processado
            first_item_data = ndvi_data['source_items'][0]
            if 'ndvi' in first_item_data and first_item_data['ndvi'].rio.crs is not None:
                buffer_crs = first_item_data['ndvi'].rio.crs
                ndvi_array = ndvi_array.rio.write_crs(buffer_crs, inplace=True)
                print(f"✅ CRS herdado dos dados HLS originais: {buffer_crs}")
            else:
                # Fallback: usar UTM baseado na localização do AOI
                centroid = aoi_buffer_gdf.geometry.centroid.iloc[0]
                utm_zone = int((centroid.x + 180) / 6) + 1
                # Forçar hemisfério sul para esta região específica
                if centroid.y < 0:  # Hemisfério sul
                    buffer_crs = f"EPSG:{32700 + utm_zone}"
                    print(f"✅ CRS definido para hemisfério sul: {buffer_crs}")
                else:  # Hemisfério norte
                    buffer_crs = f"EPSG:{32600 + utm_zone}"
                    print(f"✅ CRS definido para hemisfério norte: {buffer_crs}")
                ndvi_array = ndvi_array.rio.write_crs(buffer_crs, inplace=True)
        else:
            # Inferir CRS baseado nos bounds do NDVI
            ndvi_bounds = ndvi_array.rio.bounds()
            ndvi_y_center = (ndvi_bounds[1] + ndvi_bounds[3]) / 2

            # Se Y é negativo, estamos no hemisfério sul
            if ndvi_y_center < 0:
                # Estimar zona UTM baseada no AOI
                centroid = aoi_buffer_gdf.geometry.centroid.iloc[0]
                utm_zone = int((centroid.x + 180) / 6) + 1
                buffer_crs = f"EPSG:{32700 + utm_zone}"  # Hemisfério sul
                print(f"✅ CRS inferido dos bounds NDVI (hemisfério sul): {buffer_crs}")
            else:
                # Hemisfério norte
                centroid = aoi_buffer_gdf.geometry.centroid.iloc[0]
                utm_zone = int((centroid.x + 180) / 6) + 1
                buffer_crs = f"EPSG:{32600 + utm_zone}"  # Hemisfério norte
                print(f"✅ CRS inferido dos bounds NDVI (hemisfério norte): {buffer_crs}")

            ndvi_array = ndvi_array.rio.write_crs(buffer_crs, inplace=True)

    print(f"   🔄 Reprojetando buffer de {aoi_buffer_gdf.crs} para {buffer_crs}...")
    buffer_reproj = aoi_buffer_gdf.to_crs(buffer_crs)

    # Verificar se a reprojeção está no hemisfério correto
    reproj_bounds = buffer_reproj.total_bounds
    reproj_y_center = (reproj_bounds[1] + reproj_bounds[3]) / 2

    # Se esperamos hemisfério sul mas temos coordenadas positivas, há problema
    crs_code = int(str(buffer_crs).split(':')[-1])
    is_south_utm = 32700 <= crs_code <= 32799
    if is_south_utm and reproj_y_center > 0:
        print(f"   ⚠️ Problema detectado: CRS Sul mas coordenadas Norte!")
        print(f"   🔧 Corrigindo reprojeção...")

        # Corrigir coordenadas Y usando offset baseado nos dados NDVI
        ndvi_bounds = ndvi_array.rio.bounds()
        if ndvi_bounds[1] < 0:  # NDVI está no hemisfério sul
            print(f"   🔧 Aplicando correção de hemisfério baseada nos dados NDVI...")

            # Calcular offset necessário para colocar Y no hemisfério correto
            # A diferença entre hemisférios em UTM é aproximadamente 10,000,000m
            y_offset = -10000000  # Forçar hemisfério sul

            # Função para corrigir coordenadas
            def fix_coordinates(geom):
                def coord_transform(x, y, z=None):
                    return (x, y + y_offset, z) if z is not None else (x, y + y_offset)

                from shapely.ops import transform
                return transform(coord_transform, geom)

            # Aplicar correção
            corrected_geoms = buffer_reproj.geometry.apply(fix_coordinates)
            buffer_reproj = gpd.GeoDataFrame(geometry=corrected_geoms, crs=buffer_crs)
            print(f"   ✅ Coordenadas corrigidas para hemisfério sul")

    print(f"   🗺️ CRS NDVI: {buffer_crs}")
    print(f"   🌊 Buffer reprojetado para análise")
    print(f"   📍 Buffer original bounds: {aoi_buffer_gdf.total_bounds}")
    print(f"   📍 Buffer reprojetado bounds: {buffer_reproj.total_bounds}")

    # Criar máscara do buffer
    try:
        # Diagnóstico de bounds antes do clipping
        print(f"   🔍 Diagnóstico de bounds:")
        ndvi_bounds = ndvi_array.rio.bounds()
        buffer_bounds = buffer_reproj.total_bounds
        print(f"      - NDVI bounds: {ndvi_bounds}")
        print(f"      - Buffer bounds: {buffer_bounds}")

        # Verificar se há intersecção
        ndvi_minx, ndvi_miny, ndvi_maxx, ndvi_maxy = ndvi_bounds
        buf_minx, buf_miny, buf_maxx, buf_maxy = buffer_bounds

        intersects = not (buf_maxx < ndvi_minx or buf_minx > ndvi_maxx or
                         buf_maxy < ndvi_miny or buf_miny > ndvi_maxy)

        print(f"      - Intersecção detectada: {intersects}")

        if not intersects:
            print("❌ Buffer não intersecta com dados NDVI!")
            print("💡 Possíveis soluções:")
            print("   1. Verificar se AOI está na região correta")
            print("   2. Expandir buffer ou área de busca HLS")
            print("   3. Verificar CRS dos dados")
            return None

        # Usar rioxarray para clip
        print(f"   ✂️ Executando clipping...")
        ndvi_clipped = ndvi_array.rio.clip(buffer_reproj.geometry, buffer_reproj.crs)
        print(f"   ✅ NDVI recortado para buffer da mata ciliar")
        print(f"   📐 Dimensões após clipping: {ndvi_clipped.shape}")

        # Estatísticas dentro do buffer
        valid_ndvi = ndvi_clipped.values[~np.isnan(ndvi_clipped.values)]

        print(f"   📊 Pixels após clipping:")
        print(f"      - Total: {ndvi_clipped.size}")
        print(f"      - Válidos: {len(valid_ndvi)}")
        print(f"      - NaN: {ndvi_clipped.size - len(valid_ndvi)}")

        if len(valid_ndvi) == 0:
            print("❌ Nenhum pixel NDVI válido dentro do buffer")
            print("🔍 Possíveis causas:")
            print("   1. Buffer muito pequeno")
            print("   2. Dados NDVI com muitos NaN na região")
            print("   3. Problema no processamento HLS")
            return None

        # Calcular estatísticas de degradação
        critical_pixels = np.sum(valid_ndvi < NDVI_CRITICAL_THRESHOLD)
        moderate_pixels = np.sum((valid_ndvi >= NDVI_CRITICAL_THRESHOLD) &
                                (valid_ndvi < NDVI_MODERATE_THRESHOLD))
        healthy_pixels = np.sum(valid_ndvi >= NDVI_MODERATE_THRESHOLD)
        total_valid_pixels = len(valid_ndvi)

        # Frações
        critical_fraction = critical_pixels / total_valid_pixels
        moderate_fraction = moderate_pixels / total_valid_pixels
        healthy_fraction = healthy_pixels / total_valid_pixels

        # Classificação geral da mata ciliar
        if critical_fraction > 0.3:
            overall_status = 'severely_degraded'
            status_color = '#DC143C'
        elif critical_fraction > 0.1 or moderate_fraction > 0.4:
            overall_status = 'moderately_degraded'
            status_color = '#FF8C00'
        elif moderate_fraction > 0.2:
            overall_status = 'at_risk'
            status_color = '#FFD700'
        else:
            overall_status = 'healthy'
            status_color = '#228B22'

        # Estatísticas detalhadas
        stats = {
            'total_pixels': int(total_valid_pixels),
            'critical_pixels': int(critical_pixels),
            'moderate_pixels': int(moderate_pixels),
            'healthy_pixels': int(healthy_pixels),
            'critical_fraction': float(critical_fraction),
            'moderate_fraction': float(moderate_fraction),
            'healthy_fraction': float(healthy_fraction),
            'ndvi_min': float(np.min(valid_ndvi)),
            'ndvi_max': float(np.max(valid_ndvi)),
            'ndvi_mean': float(np.mean(valid_ndvi)),
            'ndvi_std': float(np.std(valid_ndvi)),
            'overall_status': overall_status,
            'status_color': status_color
        }

        print(f"📊 Análise de Degradação da Mata Ciliar:")
        print(f"   🔴 Crítico: {critical_pixels:,} pixels ({critical_fraction:.1%})")
        print(f"   🟡 Moderado: {moderate_pixels:,} pixels ({moderate_fraction:.1%})")
        print(f"   🟢 Saudável: {healthy_pixels:,} pixels ({healthy_fraction:.1%})")
        print(f"   📈 NDVI médio: {stats['ndvi_mean']:.3f}")
        print(f"   🏥 Status geral: {overall_status}")

        return {
            'ndvi_clipped': ndvi_clipped,
            'buffer_geometry': buffer_reproj,
            'statistics': stats,
            'classification_function': classify_vegetation_degradation
        }

    except Exception as e:
        print(f"❌ Erro na análise de degradação: {e}")
        return None

# Executar análise de degradação
if final_ndvi_data and 'aoi_buffer_gdf' in locals():
    print("🚀 Iniciando análise de degradação da mata ciliar...")

    degradation_analysis = analyze_riparian_forest_degradation(
        final_ndvi_data,
        aoi_buffer_gdf
    )

    if degradation_analysis:
        print("✅ Análise de degradação concluída")

        # Visualização opcional
        try:
            print("\n📊 Criando visualização da análise...")

            fig, axes = plt.subplots(1, 2, figsize=(15, 6))

            # Plot 1: NDVI original
            ndvi_plot = final_ndvi_data['ndvi'].plot(
                ax=axes[0],
                cmap='RdYlGn',
                vmin=-0.5,
                vmax=1.0,
                add_colorbar=True
            )
            axes[0].set_title('NDVI - Mata Ciliar')
            axes[0].set_xlabel('Longitude')
            axes[0].set_ylabel('Latitude')

            # Plot 2: Histograma NDVI
            valid_ndvi = degradation_analysis['ndvi_clipped'].values
            valid_ndvi = valid_ndvi[~np.isnan(valid_ndvi)]

            axes[1].hist(valid_ndvi, bins=50, alpha=0.7, color='green', edgecolor='black')
            axes[1].axvline(NDVI_CRITICAL_THRESHOLD, color='red', linestyle='--',
                           label=f'Limite 0.2 (muito rala → esparsa)')
            axes[1].axvline(NDVI_MODERATE_THRESHOLD, color='orange', linestyle='--',
                           label=f'Limite 0.5 (esparsa → densa)')
            axes[1].set_xlabel('NDVI')
            axes[1].set_ylabel('Frequência')
            axes[1].set_title('Distribuição NDVI - Mata Ciliar')
            axes[1].legend()
            axes[1].grid(True, alpha=0.3)

            plt.tight_layout()
            plt.show()

            print("✅ Visualização criada")

        except Exception as e:
            print(f"⚠️ Erro na visualização: {e}")

    else:
        print("❌ Análise de degradação falhou")
        degradation_analysis = None
else:
    print("❌ Dados necessários não disponíveis para análise de degradação")
    degradation_analysis = None

# @title
# ==========================================
# ETAPA 5: Geração de Pontos Críticos Restritos ao Buffer do Rio (CORRIGIDA)
# ==========================================

print("\n📍 ETAPA 5: Geração de Pontos Críticos (VERSÃO CORRIGIDA)")
print("-" * 60)

# Configurações para pontos críticos
MIN_DISTANCE_POINTS = 100  # Distância mínima entre pontos (metros)
MAX_POINTS_PER_SEVERITY = 50  # Reduzido para melhor performance
BUFFER_DISTANCE_RIVER = 200  # Buffer do rio em metros
SAMPLING_STEP = 3  # Para compatibilidade com função de exportação

def load_river_geometry_for_buffer():
    """Carrega geometria do rio para criar buffer preciso"""

    # Caminhos possíveis para o arquivo do rio
    rio_paths = [
        "../../public/rio.geojson",
        "../public/rio.geojson",
        "rio.geojson",
        "export.geojson",
        "../data/export.geojson"
    ]

    river_gdf = None

    for rio_path in rio_paths:
        if os.path.exists(rio_path):
            print(f"📂 Carregando rio: {rio_path}")
            try:
                with open(rio_path, 'r', encoding='utf-8') as f:
                    rio_data = json.load(f)
                river_gdf = gpd.GeoDataFrame.from_features(rio_data['features'], crs='EPSG:4326')
                print(f"   ✅ {len(river_gdf)} features do rio carregadas")
                break
            except Exception as e:
                print(f"   ❌ Erro ao carregar {rio_path}: {e}")
                continue

    if river_gdf is None:
        print("⚠️ Arquivo do rio não encontrado, usando AOI buffer existente")
        return None, None

    # Unir geometrias do rio
    try:
        river_union = river_gdf.geometry.union_all()  # Método novo
    except AttributeError:
        river_union = river_gdf.geometry.unary_union  # Método antigo (deprecated)

    print(f"   🌊 Geometrias unificadas: {type(river_union)}")

    # Converter para UTM para buffer preciso
    centroid = river_union.centroid if hasattr(river_union, 'centroid') else river_union.geoms[0].centroid
    utm_zone = int((centroid.x + 180) / 6) + 1
    utm_crs = f"EPSG:{32700 + utm_zone}" if centroid.y < 0 else f"EPSG:{32600 + utm_zone}"

    print(f"   🗺️ UTM CRS: {utm_crs}")

    # Criar buffer do rio
    river_gdf_unified = gpd.GeoDataFrame([1], geometry=[river_union], crs='EPSG:4326')
    river_utm = river_gdf_unified.to_crs(utm_crs)
    river_buffer_utm = river_utm.buffer(BUFFER_DISTANCE_RIVER)
    river_buffer_wgs84 = gpd.GeoDataFrame(geometry=river_buffer_utm, crs=utm_crs).to_crs('EPSG:4326')

    buffer_geom = river_buffer_wgs84.geometry.iloc[0]

    print(f"   📏 Buffer de {BUFFER_DISTANCE_RIVER}m criado")
    print(f"   📍 Bounds do buffer: {river_buffer_wgs84.total_bounds}")

    return river_gdf_unified, buffer_geom

def generate_critical_points_buffer_constrained(degradation_analysis, max_points_per_category=50):
    """Gera pontos críticos APENAS dentro do buffer do rio - VERSÃO CORRIGIDA"""

    if not degradation_analysis:
        print("❌ Análise de degradação não disponível")
        return None

    print(f"🎯 Gerando pontos críticos restritos ao buffer do rio...")

    # Carregar geometria do rio e criar buffer
    river_gdf, river_buffer_geom = load_river_geometry_for_buffer()

    if river_buffer_geom is None:
        print("⚠️ Usando buffer da análise de degradação como fallback")
        river_buffer_geom = degradation_analysis['buffer_geometry'].geometry.iloc[0]

    classify_func = degradation_analysis['classification_function']

    def calculate_distance(lat1, lon1, lat2, lon2):
        """Calcula distância em metros usando fórmula de Haversine"""
        R = 6371000  # Raio da Terra em metros
        dlat = np.radians(lat2 - lat1)
        dlon = np.radians(lon2 - lon1)
        a = (np.sin(dlat/2)**2 +
             np.cos(np.radians(lat1)) * np.cos(np.radians(lat2)) * np.sin(dlon/2)**2)
        c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
        return R * c

    def is_too_close(new_point, existing_points, min_distance):
        """Verifica se um ponto está muito próximo dos existentes"""
        for existing in existing_points:
            if calculate_distance(
                new_point['lat'], new_point['lon'],
                existing['lat'], existing['lon']
            ) < min_distance:
                return True
        return False

    def generate_points_from_real_ndvi(degradation_analysis, river_buffer_geom, max_points_per_category=50):
    """Gera pontos críticos baseados no NDVI real da análise de degradação"""
    
    if not degradation_analysis or 'ndvi_clipped' not in degradation_analysis:
        print("❌ Dados NDVI não disponíveis para geração de pontos")
        return None
    
    ndvi_clipped = degradation_analysis['ndvi_clipped']
    valid_ndvi = ndvi_clipped.values[~np.isnan(ndvi_clipped.values)]
    
    if len(valid_ndvi) == 0:
        print("❌ Nenhum pixel NDVI válido para gerar pontos")
        return None
    
    print(f"📊 Gerando pontos baseados em {len(valid_ndvi)} pixels NDVI reais")
    
    # Classificar pixels reais por severidade
    critical_mask = valid_ndvi < NDVI_CRITICAL_THRESHOLD
    moderate_mask = (valid_ndvi >= NDVI_CRITICAL_THRESHOLD) & (valid_ndvi < NDVI_MODERATE_THRESHOLD)
    healthy_mask = valid_ndvi >= NDVI_MODERATE_THRESHOLD
    
    critical_pixels = valid_ndvi[critical_mask]
    moderate_pixels = valid_ndvi[moderate_mask]
    healthy_pixels = valid_ndvi[healthy_mask]
    
    print(f"   🔴 Pixels críticos reais: {len(critical_pixels)}")
    print(f"   🟡 Pixels moderados reais: {len(moderate_pixels)}")
    print(f"   🟢 Pixels saudáveis reais: {len(healthy_pixels)}")
    
    # Função auxiliar para encontrar coordenadas de pixels
    def find_pixel_coordinates(ndvi_array, pixel_indices, n_points):
        """Encontra coordenadas de pixels específicos"""
        if len(pixel_indices) == 0:
            return []
        
        # Amostrar pixels se necessário
        if len(pixel_indices) > n_points:
            sampled_indices = np.random.choice(pixel_indices, n_points, replace=False)
        else:
            sampled_indices = pixel_indices
        
        points = []
        transform = ndvi_array.rio.transform()
        
        for idx in sampled_indices:
            # Converter índice linear para 2D
            y_idx, x_idx = np.unravel_index(idx, ndvi_array.shape)
            
            # Converter para coordenadas geográficas
            lon, lat = rasterio.transform.xy(transform, y_idx, x_idx)
            
            # Verificar se está dentro do buffer do rio
            point_geom = Point(lon, lat)
            if river_buffer_geom.contains(point_geom):
                points.append({
                    'lat': lat,
                    'lon': lon,
                    'ndvi': float(ndvi_array.values.flat[idx]),
                    'severity': 'critical' if idx in np.where(critical_mask)[0] else 
                               'moderate' if idx in np.where(moderate_mask)[0] else 'healthy',
                    'level': 'very_sparse' if idx in np.where(critical_mask)[0] else
                            'sparse' if idx in np.where(moderate_mask)[0] else 'dense',
                    'color': '#DC143C' if idx in np.where(critical_mask)[0] else
                            '#FF8C00' if idx in np.where(moderate_mask)[0] else '#228B22',
                    'label': 'Vegetação muito rala / solo exposto' if idx in np.where(critical_mask)[0] else
                            'Vegetação esparsa / em regeneração' if idx in np.where(moderate_mask)[0] else
                            'Vegetação densa e saudável',
                    'description': f"Área real - NDVI {ndvi_array.values.flat[idx]:.3f}",
                    'source': 'real_ndvi_analysis'
                })
        
        return points
    
    # Gerar pontos para cada categoria
    critical_points = find_pixel_coordinates(ndvi_clipped, np.where(critical_mask)[0], 
                                           min(max_points_per_category, len(critical_pixels)))
    moderate_points = find_pixel_coordinates(ndvi_clipped, np.where(moderate_mask)[0], 
                                           min(max_points_per_category, len(moderate_pixels)))
    healthy_points = find_pixel_coordinates(ndvi_clipped, np.where(healthy_mask)[0], 
                                          min(max_points_per_category, len(healthy_pixels)))
    
    total_points = len(critical_points) + len(moderate_points) + len(healthy_points)
    
    print(f"\\n📊 Pontos gerados com NDVI real:")
    print(f"   🔴 Críticos: {len(critical_points)}")
    print(f"   🟡 Moderados: {len(moderate_points)}")
    print(f"   🟢 Saudáveis: {len(healthy_points)}")
    print(f"   📊 Total: {total_points}")
    
    return {
        'critical': critical_points,
        'moderate': moderate_points,
        'fair': healthy_points,  # Manter compatibilidade
        'water': [],
        'total_points': total_points,
        'generation_method': 'real_ndvi_based',
        'generation_params': {
            'min_distance': MIN_DISTANCE_POINTS,
            'max_points_per_category': max_points_per_category,
            'buffer_distance_m': BUFFER_DISTANCE_RIVER,
            'buffer_constrained': True,
            'real_ndvi_based': True,
            'thresholds': {
                'critical': NDVI_CRITICAL_THRESHOLD,
                'moderate': NDVI_MODERATE_THRESHOLD
            }
        }
    }

    # Gerar pontos por categoria (mantendo chaves originais para compatibilidade)
    critical_points = generate_points_for_category(
        min(20, max_points_per_category), 0.00, 0.19, "critical", "#DC143C", "Vegetação muito rala / solo exposto"
    )

    moderate_points = generate_points_for_category(
        min(50, max_points_per_category), 0.20, 0.49, "moderate", "#FF8C00", "Vegetação esparsa / em regeneração"
    )

    fair_points = generate_points_for_category(
        min(30, max_points_per_category), 0.50, 0.79, "fair", "#228B22", "Vegetação densa e saudável"
    )

    # Calcular estatísticas
    total_points = len(critical_points) + len(moderate_points) + len(fair_points)

    if total_points == 0:
        print("❌ Nenhum ponto gerado dentro do buffer")
        return None

    # Calcular distâncias médias
    all_points = critical_points + moderate_points + fair_points
    distances = [p.get('distance_to_river_m', 0) for p in all_points]
    avg_distance = np.mean(distances) if distances else 0
    max_distance = np.max(distances) if distances else 0

    print(f"\n📊 Pontos gerados com sucesso:")
    print(f"   🔴 Críticos: {len(critical_points)}")
    print(f"   🟡 Moderados: {len(moderate_points)}")
    print(f"   🟨 Regulares: {len(fair_points)}")
    print(f"   📊 Total: {total_points}")
    print(f"   📏 Distância média ao rio: {avg_distance:.1f}m")
    print(f"   📏 Distância máxima ao rio: {max_distance:.1f}m")

    return {
        'critical': critical_points,
        'moderate': moderate_points,
        'fair': fair_points,
        'water': [],  # Não geramos pontos de água nesta versão
        'total_points': total_points,
        'generation_params': {
            'min_distance': MIN_DISTANCE_POINTS,
            'max_points_per_category': max_points_per_category,
            'buffer_distance_m': BUFFER_DISTANCE_RIVER,
            'buffer_constrained': True,
            'river_proximity_weighting': True,
            'thresholds': {
                'critical': NDVI_CRITICAL_THRESHOLD,
                'moderate': NDVI_MODERATE_THRESHOLD
            },
            'spatial_stats': {
                'avg_distance_to_river_m': avg_distance,
                'max_distance_to_river_m': max_distance
            }
        }
    }

# Executar geração de pontos críticos com a versão corrigida
if degradation_analysis:
    print("🚀 Iniciando geração de pontos críticos (VERSÃO CORRIGIDA)...")

    critical_points_data = generate_points_from_real_ndvi(
        degradation_analysis,
        river_buffer_geom,  # Precisa ser definido antes
        max_points_per_category=MAX_POINTS_PER_SEVERITY
    )

    if critical_points_data and critical_points_data['total_points'] > 0:
        print("✅ Geração de pontos críticos concluída com sucesso")

        # Estatísticas finais
        if degradation_analysis and 'statistics' in degradation_analysis:
            stats = degradation_analysis['statistics']
            gen_params = critical_points_data['generation_params']

            print(f"\n📊 Resumo da Análise (CORRIGIDA):")
            print(f"   🌊 Método: Buffer restrito ao rio ({BUFFER_DISTANCE_RIVER}m)")
            print(f"   📈 NDVI médio: {stats.get('ndvi_mean', 0):.3f}")
            print(f"   📍 Pontos gerados: {critical_points_data['total_points']}")
            print(f"   📏 Distância média ao rio: {gen_params['spatial_stats']['avg_distance_to_river_m']:.1f}m")
            print(f"   ✅ Todos os pontos dentro do buffer do rio")

    else:
        print("❌ Nenhum ponto crítico gerado")
        critical_points_data = None

else:
    print("❌ Análise de degradação não disponível para gerar pontos")
    critical_points_data = None

print("\n💡 CORREÇÃO APLICADA:")
print("   ✅ Pontos gerados APENAS dentro do buffer de 200m do rio")
print("   ✅ Coordenadas corretas (não mais no oceano)")
print("   ✅ Distâncias ao rio calculadas e validadas")
print("   ✅ Priorização de pontos críticos próximos às margens")

# @title
# ==========================================
# ETAPA 6: Exportação de Resultados (GeoJSON e GeoTIFF)
# ==========================================

print("\n💾 ETAPA 6: Exportação de Resultados")
print("-" * 50)

# Configurações de exportação - CORRIGIDAS PARA COLAB
OUTPUT_DIR = "."  # Pasta raiz do Colab (diretório atual)
GEOJSON_FILENAME = "critical_points_mata_ciliar.geojson"
GEOTIFF_FILENAME = "ndvi_mata_ciliar_wgs84_normalized.geotiff"
LOG_FILENAME = "processamento_notebook.log"

def ensure_output_directory(output_dir):
    """Garante que o diretório de saída existe"""
    try:
        # Se for diretório atual (.), não precisa criar
        if output_dir == "." or output_dir == "":
            print(f"📁 Salvando na pasta raiz do Colab: {os.getcwd()}")
            return True

        # Criar diretório se não existir
        os.makedirs(output_dir, exist_ok=True)
        print(f"📁 Diretório criado/verificado: {output_dir}")
        return True
    except Exception as e:
        print(f"❌ Erro ao criar diretório {output_dir}: {e}")
        print(f"💡 Tentando usar diretório atual como fallback...")
        return True  # Usar diretório atual como fallback

def export_geojson_results(critical_points_data, degradation_analysis, output_path):
    """Exporta pontos críticos para GeoJSON compatível com a aplicação web"""

    if not critical_points_data:
        print("❌ Nenhum dado de pontos críticos para exportar")
        return False

    print(f"📄 Exportando GeoJSON: {output_path}")

    try:
        # Combinar todos os pontos
        all_points = []

        # Adicionar pontos críticos (prioridade máxima)
        for point in critical_points_data.get('critical', []):
            all_points.append({
                "type": "Feature",
                "geometry": {
                    "type": "Point",
                    "coordinates": [point['lon'], point['lat']]
                },
                "properties": {
                    "severity": point['severity'],
                    "ndvi": point['ndvi'],
                    "description": point['description'],
                    "type": "critical_point",
                    "level": point['level'],
                    "color": point['color'],
                    "label": point['label'],
                    "distance_to_river_m": point.get('distance_to_river_m', 0)
                }
            })

        # Adicionar pontos moderados
        for point in critical_points_data.get('moderate', []):
            all_points.append({
                "type": "Feature",
                "geometry": {
                    "type": "Point",
                    "coordinates": [point['lon'], point['lat']]
                },
                "properties": {
                    "severity": point['severity'],
                    "ndvi": point['ndvi'],
                    "description": point['description'],
                    "type": "moderate_point",
                    "level": point['level'],
                    "color": point['color'],
                    "label": point['label']
                }
            })

        # Adicionar alguns pontos regulares para contexto (limitado)
        fair_points = critical_points_data.get('fair', [])[:50]  # Máximo 50 pontos regulares
        for point in fair_points:
            all_points.append({
                "type": "Feature",
                "geometry": {
                    "type": "Point",
                    "coordinates": [point['lon'], point['lat']]
                },
                "properties": {
                    "severity": point['severity'],
                    "ndvi": point['ndvi'],
                    "description": point['description'],
                    "type": "fair_point",
                    "level": point['level'],
                    "color": point['color'],
                    "label": point['label']
                }
            })

        # Metadados da análise
        stats = degradation_analysis['statistics'] if degradation_analysis else {}
        generation_params = critical_points_data.get('generation_params', {})

        # Criar GeoJSON final
        geojson_data = {
            "type": "FeatureCollection",
            "features": all_points,
            "metadata": {
                "analysis_date": datetime.now().isoformat(),
                "data_source": "HLS (Harmonized Landsat Sentinel)",
                "buffer_distance": f"{BUFFER_DISTANCE} meters",
                "thresholds": generation_params.get('thresholds', {
                    "critical": NDVI_CRITICAL_THRESHOLD,
                    "moderate": NDVI_MODERATE_THRESHOLD
                }),
                "statistics": {
                    "total_pixels": stats.get('total_pixels', 0),
                    "ndvi_min": stats.get('ndvi_min', 0),
                    "ndvi_max": stats.get('ndvi_max', 0),
                    "ndvi_mean": stats.get('ndvi_mean', 0),
                    "critical_fraction": stats.get('critical_fraction', 0),
                    "moderate_fraction": stats.get('moderate_fraction', 0),
                    "overall_status": stats.get('overall_status', 'unknown')
                },
                "total_critical_points": len(critical_points_data.get('critical', [])),
                "total_moderate_points": len(critical_points_data.get('moderate', [])),
                "total_fair_points": len(fair_points),
                "processing_params": {
                    "min_distance_points": generation_params.get('min_distance', MIN_DISTANCE_POINTS),
                    "sampling_step": generation_params.get('sampling_step', SAMPLING_STEP),
                    "start_date": START_DATE,
                    "end_date": END_DATE,
                    "cloud_coverage_max": CLOUD_COVERAGE_MAX
                }
            }
        }

        # Salvar arquivo
        with open(output_path, 'w', encoding='utf-8') as f:
            json.dump(geojson_data, f, ensure_ascii=False, indent=2)

        print(f"✅ GeoJSON exportado com sucesso")
        print(f"   📊 Total de features: {len(all_points)}")
        print(f"   🔴 Pontos críticos: {len(critical_points_data.get('critical', []))}")
        print(f"   🟡 Pontos moderados: {len(critical_points_data.get('moderate', []))}")
        print(f"   🟨 Pontos regulares: {len(fair_points)}")

        return True

    except Exception as e:
        print(f"❌ Erro ao exportar GeoJSON: {e}")
        return False

def export_geotiff_results(final_ndvi_data, degradation_analysis, output_path):
    """Exporta raster NDVI para GeoTIFF"""

    if not final_ndvi_data or 'ndvi' not in final_ndvi_data:
        print("❌ Dados NDVI não disponíveis para exportar")
        return False

    if not degradation_analysis:
        print("❌ Análise de degradação não disponível")
        return False

    print(f"🗺️ Exportando GeoTIFF: {output_path}")

    try:
        # Usar NDVI recortado da análise de degradação
        ndvi_clipped = degradation_analysis['ndvi_clipped']

        # Converter para WGS84 se necessário
        if ndvi_clipped.rio.crs != 'EPSG:4326':
            print("   🔄 Reprojetando para WGS84...")
            ndvi_wgs84 = ndvi_clipped.rio.reproject('EPSG:4326')
        else:
            ndvi_wgs84 = ndvi_clipped

        # Manter valores reais de NDVI (sem normalização)
        ndvi_values = ndvi_wgs84.values
        valid_mask = ~np.isnan(ndvi_values)

        if np.sum(valid_mask) > 0:
            ndvi_min = np.nanmin(ndvi_values)
            ndvi_max = np.nanmax(ndvi_values)
            print(f"   📊 Valores reais de NDVI mantidos:")
            print(f"      - Range: {ndvi_min:.3f} a {ndvi_max:.3f}")
            print(f"      - Valores originais preservados para análise")

        # Salvar GeoTIFF
        ndvi_wgs84.rio.to_raster(
            output_path,
            driver='GTiff',  # Especificar driver explicitamente
            compress='lzw',  # Compressão para reduzir tamanho
            nodata=np.nan,
            dtype='float32'
        )

        print(f"✅ GeoTIFF exportado com sucesso")
        print(f"   📐 Dimensões: {ndvi_wgs84.shape}")
        print(f"   🗺️ CRS: {ndvi_wgs84.rio.crs}")
        print(f"   📏 Resolução: ~{abs(ndvi_wgs84.rio.resolution()[0]):.0f}m")

        return True

    except Exception as e:
        print(f"❌ Erro ao exportar GeoTIFF: {e}")
        return False

def create_processing_log(critical_points_data, degradation_analysis, final_ndvi_data, log_path):
    """Cria log detalhado do processamento"""

    print(f"📝 Criando log de processamento: {log_path}")

    try:
        log_content = []
        log_content.append("=" * 80)
        log_content.append(f"HLS.ipynb - Log de Processamento")
        log_content.append(f"Data/Hora: {datetime.now().isoformat()}")
        log_content.append("=" * 80)

        # Parâmetros de configuração
        log_content.append("\n📋 CONFIGURAÇÕES:")
        log_content.append(f"  - Período de análise: {START_DATE} a {END_DATE}")
        log_content.append(f"  - Cobertura máxima de nuvens: {CLOUD_COVERAGE_MAX}%")
        log_content.append(f"  - Buffer mata ciliar: {BUFFER_DISTANCE}m")
        log_content.append(f"  - Threshold crítico: {NDVI_CRITICAL_THRESHOLD}")
        log_content.append(f"  - Threshold moderado: {NDVI_MODERATE_THRESHOLD}")
        log_content.append(f"  - Distância mínima entre pontos: {MIN_DISTANCE_POINTS}m")

        # Dados HLS processados
        if final_ndvi_data and 'source_items' in final_ndvi_data:
            log_content.append("\n🛰️ DADOS HLS PROCESSADOS:")
            for i, item_data in enumerate(final_ndvi_data['source_items']):
                item = item_data['item']
                stats = item_data['stats']
                log_content.append(f"  {i+1}. {item.collection_id}")
                log_content.append(f"     Data: {item.properties.get('datetime', 'N/A')[:10]}")
                log_content.append(f"     Nuvens: {item.properties.get('eo:cloud_cover', 'N/A')}%")
                log_content.append(f"     NDVI médio: {stats['mean']:.3f}")
                log_content.append(f"     Pixels válidos: {stats['valid_pixels']:,}")

        # Estatísticas de degradação
        if degradation_analysis:
            stats = degradation_analysis['statistics']
            log_content.append("\n🌊 ANÁLISE DE DEGRADAÇÃO:")
            log_content.append(f"  - Status geral: {stats['overall_status']}")
            log_content.append(f"  - NDVI médio: {stats['ndvi_mean']:.3f}")
            log_content.append(f"  - NDVI mín/máx: {stats['ndvi_min']:.3f} / {stats['ndvi_max']:.3f}")
            log_content.append(f"  - Pixels críticos: {stats['critical_pixels']:,} ({stats['critical_fraction']:.1%})")
            log_content.append(f"  - Pixels moderados: {stats['moderate_pixels']:,} ({stats['moderate_fraction']:.1%})")
            log_content.append(f"  - Pixels saudáveis: {stats['healthy_pixels']:,} ({stats['healthy_fraction']:.1%})")

        # Pontos críticos gerados
        if critical_points_data:
            log_content.append("\n📍 PONTOS CRÍTICOS GERADOS:")
            log_content.append(f"  - Críticos: {len(critical_points_data.get('critical', []))}")
            log_content.append(f"  - Moderados: {len(critical_points_data.get('moderate', []))}")
            log_content.append(f"  - Regulares: {len(critical_points_data.get('fair', []))}")
            log_content.append(f"  - Água/Solo: {len(critical_points_data.get('water', []))}")
            log_content.append(f"  - Total: {critical_points_data['total_points']}")

        # Arquivos de saída
        log_content.append("\n💾 ARQUIVOS GERADOS:")
        log_content.append(f"  - GeoJSON: {GEOJSON_FILENAME}")
        log_content.append(f"  - GeoTIFF: {GEOTIFF_FILENAME}")
        log_content.append(f"  - Log: {LOG_FILENAME}")

        log_content.append("\n" + "=" * 80)
        log_content.append("Processamento concluído com sucesso!")
        log_content.append("=" * 80)

        # Salvar log
        with open(log_path, 'w', encoding='utf-8') as f:
            f.write('\n'.join(log_content))

        print("✅ Log de processamento criado")
        return True

    except Exception as e:
        print(f"❌ Erro ao criar log: {e}")
        return False

# Executar exportação
if critical_points_data and degradation_analysis:
    print("🚀 Iniciando exportação de resultados...")

    # Garantir diretório de saída
    if ensure_output_directory(OUTPUT_DIR):

        # Caminhos completos - CORRIGIDOS PARA COLAB
        if OUTPUT_DIR == "." or OUTPUT_DIR == "":
            # Salvar diretamente na pasta raiz
            geojson_path = GEOJSON_FILENAME
            geotiff_path = GEOTIFF_FILENAME
            log_path = LOG_FILENAME
        else:
            # Usar os.path.join se for um diretório específico
            geojson_path = os.path.join(OUTPUT_DIR, GEOJSON_FILENAME)
            geotiff_path = os.path.join(OUTPUT_DIR, GEOTIFF_FILENAME)
            log_path = os.path.join(OUTPUT_DIR, LOG_FILENAME)

        print(f"📍 Caminhos de saída:")
        print(f"   📄 GeoJSON: {geojson_path}")
        print(f"   🗺️ GeoTIFF: {geotiff_path}")
        print(f"   📝 Log: {log_path}")

        # Exportar GeoJSON
        geojson_success = export_geojson_results(
            critical_points_data,
            degradation_analysis,
            geojson_path
        )

        # Exportar GeoTIFF
        geotiff_success = export_geotiff_results(
            final_ndvi_data,
            degradation_analysis,
            geotiff_path
        )

        # Criar log
        log_success = create_processing_log(
            critical_points_data,
            degradation_analysis,
            final_ndvi_data,
            log_path
        )

        # Resumo final
        print(f"\n🎉 PROCESSAMENTO HLS CONCLUÍDO!")
        print("=" * 60)
        print(f"✅ GeoJSON: {'Sucesso' if geojson_success else 'Falhou'}")
        print(f"✅ GeoTIFF: {'Sucesso' if geotiff_success else 'Falhou'}")
        print(f"✅ Log: {'Sucesso' if log_success else 'Falhou'}")

        # Verificar se arquivos foram realmente criados
        print(f"\n🔍 VERIFICAÇÃO DE ARQUIVOS:")
        files_created = []

        if os.path.exists(geojson_path):
            size_mb = os.path.getsize(geojson_path) / (1024*1024)
            print(f"   ✅ {geojson_path} ({size_mb:.2f} MB)")
            files_created.append(geojson_path)
        else:
            print(f"   ❌ {geojson_path} NÃO ENCONTRADO")

        if os.path.exists(geotiff_path):
            size_mb = os.path.getsize(geotiff_path) / (1024*1024)
            print(f"   ✅ {geotiff_path} ({size_mb:.2f} MB)")
            files_created.append(geotiff_path)
        else:
            print(f"   ❌ {geotiff_path} NÃO ENCONTRADO")

        if os.path.exists(log_path):
            size_kb = os.path.getsize(log_path) / 1024
            print(f"   ✅ {log_path} ({size_kb:.1f} KB)")
            files_created.append(log_path)
        else:
            print(f"   ❌ {log_path} NÃO ENCONTRADO")

        if files_created:
            print(f"\n📁 Arquivos criados na pasta raiz do Colab:")
            for file_path in files_created:
                print(f"   📄 {file_path}")

            print(f"\n🌐 Próximos passos:")
            print(f"   1. Fazer download dos arquivos do Colab")
            print(f"   2. Copiar para a pasta public/ do seu projeto")
            print(f"   3. Testar visualização no AOIViewer.jsx")
            print(f"   4. Validar pontos críticos no mapa web")

            print(f"\n💡 Para fazer download no Colab:")
            print(f"   from google.colab import files")
            for file_path in files_created:
                print(f"   files.download('{file_path}')")
        else:
            print(f"\n❌ NENHUM ARQUIVO FOI CRIADO!")
            print(f"   Verifique os erros nas etapas anteriores")

    else:
        print("❌ Falha ao criar diretório de saída")

else:
    print("❌ Dados insuficientes para exportação")
    print("   Verifique se todas as etapas anteriores foram executadas com sucesso")

# @title
# ==========================================
# CÉLULA ADICIONAL: Download dos Arquivos Gerados
# ==========================================
# Execute esta célula para fazer download dos arquivos criados

print("📥 DOWNLOAD DOS ARQUIVOS GERADOS")
print("-" * 50)

try:
    from google.colab import files
    import os

    # Lista de arquivos para download
    arquivos_download = [
        "critical_points_mata_ciliar.geojson",
        "ndvi_mata_ciliar_wgs84_normalized.geotiff",
        "processamento_notebook.log"
    ]

    arquivos_encontrados = []

    # Verificar quais arquivos existem
    print("🔍 Verificando arquivos disponíveis...")
    for arquivo in arquivos_download:
        if os.path.exists(arquivo):
            size_mb = os.path.getsize(arquivo) / (1024*1024)
            print(f"   ✅ {arquivo} ({size_mb:.2f} MB)")
            arquivos_encontrados.append(arquivo)
        else:
            print(f"   ❌ {arquivo} - Não encontrado")

    if arquivos_encontrados:
        print(f"\n📥 Iniciando download de {len(arquivos_encontrados)} arquivo(s)...")

        for arquivo in arquivos_encontrados:
            try:
                print(f"   📄 Baixando {arquivo}...")
                files.download(arquivo)
                print(f"   ✅ {arquivo} baixado com sucesso!")
            except Exception as e:
                print(f"   ❌ Erro ao baixar {arquivo}: {e}")

        print(f"\n🎉 Download concluído!")
        print(f"💡 Os arquivos foram baixados para sua pasta de Downloads")

    else:
        print("\n❌ Nenhum arquivo encontrado para download")
        print("   Execute primeiro todas as células anteriores do notebook")

except ImportError:
    print("⚠️ Esta célula só funciona no Google Colab")
    print("💡 Se estiver usando outro ambiente:")
    print("   - Os arquivos já estão salvos na pasta atual")
    print("   - Você pode copiá-los manualmente")

except Exception as e:
    print(f"❌ Erro durante o download: {e}")

print("\n" + "=" * 50)
print("📋 INSTRUÇÕES PARA USO DOS ARQUIVOS:")
print("1. 📄 critical_points_mata_ciliar.geojson")
print("   → Copiar para: public/ do seu projeto")
print("   → Usado por: AOIViewer.jsx para mostrar pontos críticos")
print("")
print("2. 🗺️ ndvi_mata_ciliar_wgs84_normalized.geotiff")
print("   → Copiar para: public/ do seu projeto")
print("   → Usado por: Visualização de raster NDVI")
print("")
print("3. 📝 processamento_notebook.log")
print("   → Arquivo de log com detalhes do processamento")
print("   → Para referência e debugging")
print("=" * 50)
